In [39]:
import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib widget

In [40]:
# Load the Excel file into a DataFrame
NrSubtype='L1' #'L1' or 'All'
analysisfile='VigilanceStates_GrandGlobalAB'
directory="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/Analysis_VigStates/"
combined_df = pd.read_excel(f'{directory}{NrSubtype}_{analysisfile}.xlsx', index_col=0)

In [41]:
combined_df

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,Substate,SubstateNumber,DurationSubstate,CalciumActivity,...,Avg_CalciumActivity,Avg_AUC_calcium,SpikeActivity,AUC_spike,Avg_SpikeActivity,Avg_AUC_spike,NormalizedAUC_calcium,NormalizedAUC_spike,Unit_ID,Substate_ID
0,BlackLinesOK,session1,NaN,3,0,3,NREM,0,21.20,0.000000,...,0.342322,6104.975060,0.000000,0.00000,0.006326,112.814547,0.000000,21.20000,BlackLinesOK3,BlackLinesOKsession1NREM0
1,BlackLinesOK,session1,NaN,3,0,3,Wake,1,15.00,0.000000,...,0.342322,6104.975060,0.000000,0.00000,0.006326,112.814547,0.000000,15.00000,BlackLinesOK3,BlackLinesOKsession1Wake1
2,BlackLinesOK,session1,NaN,3,0,3,NREM,2,30.00,0.000000,...,0.342322,6104.975060,0.000000,0.00000,0.006326,112.814547,0.000000,30.00000,BlackLinesOK3,BlackLinesOKsession1NREM2
3,BlackLinesOK,session1,NaN,3,0,3,Wake,3,5.00,0.000000,...,0.342322,6104.975060,0.000000,0.00000,0.006326,112.814547,0.000000,5.00000,BlackLinesOK3,BlackLinesOKsession1Wake3
4,BlackLinesOK,session1,NaN,3,0,3,NREM,4,30.00,0.000000,...,0.342322,6104.975060,0.000000,0.00000,0.006326,112.814547,0.000000,30.00000,BlackLinesOK3,BlackLinesOKsession1NREM4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10113,RedLinesOK,session5,NaN,75,8,108,REM,22,20.00,0.064148,...,0.204801,2258.335837,0.000000,0.00000,0.002479,27.340362,1.271885,20.00000,RedLinesOK75,RedLinesOKsession5REM22
10114,RedLinesOK,session5,NaN,75,8,108,NREM,23,15.00,0.654096,...,0.204801,2258.335837,0.009295,2.77919,0.002479,27.340362,13.024277,17.77919,RedLinesOK75,RedLinesOKsession5NREM23
10115,RedLinesOK,session5,NaN,75,8,108,REM,24,15.00,0.109992,...,0.204801,2258.335837,0.000000,0.00000,0.002479,27.340362,2.178519,15.00000,RedLinesOK75,RedLinesOKsession5REM24
10116,RedLinesOK,session5,NaN,75,8,108,NREM,25,10.00,0.004004,...,0.204801,2258.335837,0.000000,0.00000,0.002479,27.340362,0.079101,10.00000,RedLinesOK75,RedLinesOKsession5NREM25


In [42]:
combined_df = combined_df[combined_df['DurationSubstate'] >= 20]

In [43]:
combined_df

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,Substate,SubstateNumber,DurationSubstate,CalciumActivity,...,Avg_CalciumActivity,Avg_AUC_calcium,SpikeActivity,AUC_spike,Avg_SpikeActivity,Avg_AUC_spike,NormalizedAUC_calcium,NormalizedAUC_spike,Unit_ID,Substate_ID
0,BlackLinesOK,session1,NaN,3,0,3,NREM,0,21.2,0.000000,...,0.342322,6104.975060,0.000000,0.000000,0.006326,112.814547,0.000000,21.200000,BlackLinesOK3,BlackLinesOKsession1NREM0
2,BlackLinesOK,session1,NaN,3,0,3,NREM,2,30.0,0.000000,...,0.342322,6104.975060,0.000000,0.000000,0.006326,112.814547,0.000000,30.000000,BlackLinesOK3,BlackLinesOKsession1NREM2
4,BlackLinesOK,session1,NaN,3,0,3,NREM,4,30.0,0.000000,...,0.342322,6104.975060,0.000000,0.000000,0.006326,112.814547,0.000000,30.000000,BlackLinesOK3,BlackLinesOKsession1NREM4
5,BlackLinesOK,session1,NaN,3,0,3,REM,5,20.0,0.000000,...,0.342322,6104.975060,0.000000,0.000000,0.006326,112.814547,0.000000,20.000000,BlackLinesOK3,BlackLinesOKsession1REM5
6,BlackLinesOK,session1,NaN,3,0,3,NREM,6,45.0,0.000000,...,0.342322,6104.975060,0.000000,0.000000,0.006326,112.814547,0.000000,45.000000,BlackLinesOK3,BlackLinesOKsession1NREM6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10106,RedLinesOK,session5,NaN,75,8,108,N2,15,60.0,0.000000,...,0.204801,2258.335837,0.000000,0.000000,0.002479,27.340362,0.000000,60.000000,RedLinesOK75,RedLinesOKsession5N215
10109,RedLinesOK,session5,NaN,75,8,108,NREM,18,35.0,0.283804,...,0.204801,2258.335837,0.003234,2.260825,0.002479,27.340362,5.663463,37.260825,RedLinesOK75,RedLinesOKsession5NREM18
10111,RedLinesOK,session5,NaN,75,8,108,REM,20,30.0,1.497306,...,0.204801,2258.335837,0.021094,12.635161,0.002479,27.340362,29.865383,42.635161,RedLinesOK75,RedLinesOKsession5REM20
10112,RedLinesOK,session5,NaN,75,8,108,NREM,21,25.0,1.702298,...,0.204801,2258.335837,0.017674,8.819225,0.002479,27.340362,33.935860,33.819225,RedLinesOK75,RedLinesOKsession5NREM21


AUC calcium

In [44]:
def max_column_name(row):
    return row.idxmax()

def normalize_row(row):
    max_col = row.idxmax()  # Find the column with the maximum value
    max_val = row[max_col]  # Get the maximum value
    return row / max_val   # Normalize the row by dividing by the maximum value

In [45]:
resultNormalizedAUC_calcium_perUnit = combined_df.pivot_table(index='Unit_ID', columns='Substate', values='NormalizedAUC_calcium', aggfunc='mean')
desired_order = ['Wake', 'N2','NREM', 'REM']  # Example order
resultNormalizedAUC_calcium_perUnit = resultNormalizedAUC_calcium_perUnit[desired_order]

max_values_per_row = resultNormalizedAUC_calcium_perUnit.max(axis=1)
Diff_resultNormalizedAUC_calcium_perUnit = (resultNormalizedAUC_calcium_perUnit.sub(max_values_per_row, axis=0) +100)
Diff_resultNormalizedAUC_calcium_perUnit['Activated_by'] = resultNormalizedAUC_calcium_perUnit.apply(max_column_name, axis=1)

# Add a new column with the column name that has the maximum value
resultNormalizedAUC_calcium_perUnit['Activated_by'] = resultNormalizedAUC_calcium_perUnit.apply(max_column_name, axis=1)

# Save  df
filenameOut = f'{directory}{NrSubtype}_VigSt_AUC_perUnitAB_N_min20s.xlsx'
writer = pd.ExcelWriter(filenameOut)
resultNormalizedAUC_calcium_perUnit.to_excel(writer)
writer.close()

# Save  df
filenameOut = f'{directory}{NrSubtype}_DiffVigSt_AUC_perUnitAB_N_min20s.xlsx'
writer = pd.ExcelWriter(filenameOut)
Diff_resultNormalizedAUC_calcium_perUnit.to_excel(writer)
writer.close()


In [46]:
proportions = resultNormalizedAUC_calcium_perUnit['Activated_by'].value_counts(normalize=True)*100
proportions

Activated_by
Wake    58.602151
REM     23.655914
NREM     9.139785
N2       8.602151
Name: proportion, dtype: float64

In [47]:
resultNormalizedAUC_calcium_perMouse = combined_df.pivot_table(index='Mice', columns='Substate', values='NormalizedAUC_calcium', aggfunc='mean')
desired_order = ['Wake', 'N2','NREM', 'REM']  # Example order
resultNormalizedAUC_calcium_perMouse = resultNormalizedAUC_calcium_perMouse[desired_order]


filenameOut = f'{directory}{NrSubtype}_VigSt_AUC_perMouseAB_min20s.xlsx'
writer = pd.ExcelWriter(filenameOut)
resultNormalizedAUC_calcium_perMouse.to_excel(writer)
writer.close()

Spike Activity 

In [48]:
resultSpikeActivity_perUnit = combined_df.pivot_table(index='Unit_ID', columns='Substate', values='SpikeActivity', aggfunc='mean')
desired_order = ['Wake','N2', 'NREM', 'REM']  # Example order
resultSpikeActivity_perUnit = resultSpikeActivity_perUnit[desired_order]

max_values_per_row = resultSpikeActivity_perUnit.max(axis=1)
Diff_resultSpikeActivity_perUnit = (resultSpikeActivity_perUnit.sub(max_values_per_row, axis=0) +100)
Diff_resultSpikeActivity_perUnit['Activated_by'] = resultSpikeActivity_perUnit.apply(max_column_name, axis=1)


# Add a new column with the column name that has the maximum value
resultSpikeActivity_perUnit['Activated_by'] = resultSpikeActivity_perUnit.apply(max_column_name, axis=1)


# Save  df
filenameOut = f'{directory}{NrSubtype}_VigSt_Spike_perUnitAB_min20s.xlsx'
writer = pd.ExcelWriter(filenameOut)
resultSpikeActivity_perUnit.to_excel(writer)
writer.close()


# Save  df
filenameOut = f'{directory}{NrSubtype}_DiffVigSt_Spike_perUnitAB_N_min20s.xlsx'
writer = pd.ExcelWriter(filenameOut)
Diff_resultSpikeActivity_perUnit.to_excel(writer)
writer.close()

In [49]:
resultSpikeActivity_perMouse = combined_df.pivot_table(index='Mice', columns='Substate', values='SpikeActivity', aggfunc='mean')
desired_order = ['Wake', 'N2', 'NREM','REM']  # Example order
resultSpikeActivity_perMouse = resultSpikeActivity_perMouse[desired_order]

filenameOut = f'{directory}{NrSubtype}_VigSt_Spike_perMouseAB_min20s.xlsx'
writer = pd.ExcelWriter(filenameOut)
resultSpikeActivity_perMouse.to_excel(writer)
writer.close()

In [50]:
proportions = resultSpikeActivity_perUnit['Activated_by'].value_counts(normalize=True)*100
proportions

Activated_by
Wake    53.763441
REM     22.580645
NREM    12.903226
N2      10.752688
Name: proportion, dtype: float64